## Functions

In [1]:
function hello1(name)
    println("What's up $name ?")
end

hello1 (generic function with 1 method)

In [2]:
hello1("doc")

What's up doc ?


In [3]:
function collatz(x)
    if Bool(x & 1)
        3 * x + 1
    else
        x / 2
    end
end

collatz (generic function with 1 method)

In [4]:
collatz(10)

5.0

In [5]:
function f1(x)
    println(x)
    x+1
end
f1(4)

4


5

In [6]:

@code_llvm optimize=false f1(4)


; Function f1
; Location: In[5]:2
define i64 @julia_f1_36307(i64) {
top:
  %1 = call %jl_value_t*** @julia.ptls_states()
  %2 = bitcast %jl_value_t*** %1 to %jl_value_t addrspace(10)**
  %3 = getelementptr inbounds %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %2, i64 3
  %4 = bitcast %jl_value_t addrspace(10)** %3 to i64**
  %5 = load i64*, i64** %4
  call void @julia_println_36308(i64 %0)
; Location: In[5]:3
; Function +; {
; Location: int.jl:53
  %6 = add i64 %0, 1
;}
  ret i64 %6
}


In [7]:
@code_native f1(4)

	.section	__TEXT,__text,regular,pure_instructions
; Function f1 {
; Location: In[5]:2
	pushl	%ebx
	decl	%eax
	movl	%edi, %ebx
	decl	%eax
	movl	$170603792, %eax        ## imm = 0xA2B3510
	addl	%eax, (%eax)
	addb	%al, (%eax)
	calll	*%eax
; Location: In[5]:3
; Function +; {
; Location: int.jl:53
	decl	%eax
	leal	1(%ebx), %eax
;}
	popl	%ebx
	retl
	nopw	%cs:(%eax,%eax)
;}


In [8]:
hello2(name) = println("What's up $name ?")

hello2 (generic function with 1 method)

In [9]:
f2(x) = x ^2

f2 (generic function with 1 method)

In [10]:
f2(4)

16

In [11]:
whatsup = name -> println("What's up $name ?")

#3 (generic function with 1 method)

In [12]:
whatsup("doc")

What's up doc ?


In [13]:
f3 = x ->  x^3

#5 (generic function with 1 method)

In [14]:
f3(4)

64

## Duck-typing

In [15]:
whatsup(4308)

What's up 4308 ?


In [41]:
A = rand(3,3)
try
    f1(A)
catch e
    println(e)
end

[0.0153863 0.792574 0.569952; 0.718287 0.612566 0.0710566; 0.754975 0.635538 0.947431]
MethodError(+, ([0.0153863 0.792574 0.569952; 0.718287 0.612566 0.0710566; 0.754975 0.635538 0.947431], 1), 0x00000000000061c6)


`v^2` for a vector is ambiguous

In [42]:
v = rand(3)

3-element Array{Float64,1}:
 0.6534089536437024
 0.9520571659451094
 0.2947126826608282

In [43]:
try
    f2(v)
catch e
    println(e)
end
    

MethodError(^, ([0.653409, 0.952057, 0.294713], 2), 0x00000000000061c6)


### Mutating vs. non-mutating functions

In [19]:
v = [3, 7, 4, 1, 4]

5-element Array{Int64,1}:
 3
 7
 4
 1
 4

In [20]:
sort(v)

5-element Array{Int64,1}:
 1
 3
 4
 4
 7

In [21]:
v

5-element Array{Int64,1}:
 3
 7
 4
 1
 4

In [22]:
sort!(v)

5-element Array{Int64,1}:
 1
 3
 4
 4
 7

In [23]:
v

5-element Array{Int64,1}:
 1
 3
 4
 4
 7

In [24]:
A = collect(1:8)
function norm( X )
    for (i,a) in enumerate(X)
        X[i] = a * a  # X Array is mofified in norm function
    end
    sqrt(sum(X))
end
norm(A)

14.2828568570857

In [25]:
A

8-element Array{Int64,1}:
  1
  4
  9
 16
 25
 36
 49
 64

### Broadcasting

In [26]:
A = [i + 3*j for j in 0:2, i in 1:3]

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

In [27]:
f = x -> x * x

f(A)

3×3 Array{Int64,2}:
  30   36   42
  66   81   96
 102  126  150

In [28]:
f.(A)

3×3 Array{Int64,2}:
  1   4   9
 16  25  36
 49  64  81

In [29]:
v = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [30]:
f.(v)

3-element Array{Int64,1}:
 1
 4
 9

In [31]:
try 
    f(v)
catch e
    println(e)
end
    

MethodError(*, ([1, 2, 3], [1, 2, 3]), 0x00000000000061bd)


In [32]:
function is_prime(n::Int)
    n < 4 && return true
    n % 2 == 0 && return false
    k = 3
    while (k*k <= n)
        n % k == 0 && return false
        k += 2
    end
    return true
end

is_prime (generic function with 1 method)

In [33]:
[i for i = 1:20 if is_prime(i)]

9-element Array{Int64,1}:
  1
  2
  3
  5
  7
 11
 13
 17
 19

In [34]:
a=1
g(x)=a+x
g(1)

2

In [35]:
a=1
function gg(x)
    x+a
end
gg(1)

2

In [36]:
a=1
function gg(x)    
    a=2
    x+a
end
gg(1), a

(3, 1)

In [37]:
function gg(x, a)
     a=2
    x+a
end
gg(1), a  

(3, 1)

# Anonymous Functions

In [38]:
x -> x^2 + 2x - 1

#13 (generic function with 1 method)

In [39]:
map(x -> x^2 + 2x - 1, [1,3,-1])

3-element Array{Int64,1}:
  2
 14
 -2

In [40]:
filter( x -> x%3==0, 1:10)

3-element Array{Int64,1}:
 3
 6
 9